## Video Load

In [ ]:
# !pip install opencv-python
# !pip install matplotlib==3.3.0

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [4]:
def load_video(path, ms=100):
    
    # Video Capture 객체 생성
    capture = cv2.VideoCapture(path)
    
    # Frame을 저장할 List 선언
    frames = []
    
    while capture.isOpened(): # Video Capture가 준비되었는지 확인
        
        run, frame = capture.read() # 다음 Frame 읽기
        
        if run: # Frame을 읽은 경우
            cv2.imshow("video", frame)
            if cv2.waitKey(ms) & 0xFF == ord('q'):
                break
        else: # 재생이 완료되어 더 이상 Frame을 읽을 수 없는 경우
            break
        
        # Frame List에 추가
        frames.append(frame)

    capture.release() # Capture 자원 반납
    cv2.destroyAllWindows() # 창 제거
    
    return np.array(frames, dtype='uint8')

In [5]:
# Video가 저장된 경로 입력
PATH = r".\pushup_sample.mp4"
# PATH = r"C:\Users\admin\Desktop\KUDIP\Video Samples\Hand Video2.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\highway.mov"
# PATH = r"C:\Users\hj\AICV\수업\디지털영상처리\Video Samples\earth.avi"

# Video 재생 및 반환 (Numpy Array)
video = load_video(PATH)

In [6]:
print(f"- Data Type: {type(video)}")
print(f"- Data Shape: {video.shape} *Frames x Height x Width x Channel")
if video.size > 0:
    print(f"- Maximum Intensity: {video.max()}")
    print(f"- Minimum Intensity: {video.min()}")
else:
    print("The array is empty, so maximum and minimum intensities cannot be determined.")

- Data Type: <class 'numpy.ndarray'>
- Data Shape: (21, 360, 640, 3) *Frames x Height x Width x Channel
- Maximum Intensity: 189
- Minimum Intensity: 0


In [ ]:
import cv2
import numpy as np

def calculate_circularity(contour):
    area = cv2.contourArea(contour)
    perimeter = cv2.arcLength(contour, True)
    circularity = (4 * np.pi * area) / (perimeter ** 2)
    return circularity

def find_largest_circularity_contour(contours):
    max_circularity = -1
    largest_contour = None
    
    for contour in contours:
        circularity = calculate_circularity(contour)
        if circularity > max_circularity:
            max_circularity = circularity
            largest_contour = contour
    
    return largest_contour

def find_center_of_contour(contour):
    ((x, y), _radius) = cv2.minEnclosingCircle(contour)
    center = (int(x), int(y))
    return center

def high_boost_filtering(image, alpha=2, kernel_size=(3, 3)):
    # Apply low-pass filtering (blur)
    blurred = cv2.GaussianBlur(image, kernel_size, 0)
    
    # Compute high-pass filtered image
    high_pass = image - blurred
    
    # Apply high-boost filtering
    high_boosted = image + alpha * high_pass
    
    # Clip values to ensure they are within the valid range [0, 255]
    high_boosted = np.clip(high_boosted, 0, 255)
    
    return high_boosted.astype(np.uint8)

def preprocess_frame(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    intensity = hsv[:, :, 2]

    boosted = high_boost_filtering(intensity)

    # 원형 커널 정의 (반지름 5)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    opening = cv2.morphologyEx(boosted, cv2.MORPH_DILATE, kernel)

    hsv[:, :, 2] = opening
    result = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)

    return result

def three_step_search(prev_frame, curr_frame, search_range=15):
    # Get the shape of the frame
    height, width = prev_frame.shape[:2]
    
    # Initialize motion vector
    motion_vector = np.zeros(2, dtype=np.uint8)
    
    # Divide the frame into blocks and initialize initial search point
    block_size = 16
    x, y = width // 2, height // 2
    
    # Calculate the initial block and search area
    prev_block = prev_frame[y-block_size//2:y+block_size//2, x-block_size//2:x+block_size//2]
    search_area = curr_frame[max(0, y-search_range):min(height, y+search_range),
                             max(0, x-search_range):min(width, x+search_range)]
    
    # Search for the best match in the search area
    min_sad = float('inf')
    for dy in range(-search_range, search_range+1, 3):
        for dx in range(-search_range, search_range+1, 3):
            if y+dy-block_size//2 < 0 or y+dy+block_size//2 >= height or x+dx-block_size//2 < 0 or x+dx+block_size//2 >= width:
                continue
            
            curr_block = curr_frame[y+dy-block_size//2:y+dy+block_size//2, x+dx-block_size//2:x+dx+block_size//2]
            sad = np.sum(np.abs(prev_block.astype(np.uint8) - curr_block.astype(np.uint8)))
            
            if sad < min_sad:
                min_sad = sad
                motion_vector[0] = dx
                motion_vector[1] = dy
    
    return motion_vector


def count_pushups(head_positions):
    count = 0
    state = 0  # 0: 시작, 1: 내려감, 2: 올라옴
    for i in range(1, len(head_positions)):
        if head_positions[i-1] is None or head_positions[i] is None:
            continue
        if state == 0 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
        elif state == 1 and head_positions[i][1] < head_positions[i-1][1]:
            count += 1
            state = 2
        elif state == 2 and head_positions[i][1] > head_positions[i-1][1]:
            state = 1
    return count

def detect_skin(img):

    #converting from gbr to hsv color space
    img_HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    #skin color range for hsv color space 
    HSV_mask = cv2.inRange(img_HSV, (0, 15, 0), (17,170,255)) 

    HSV_mask = cv2.morphologyEx(HSV_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #converting from gbr to YCbCr color space
    img_YCrCb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)

    #skin color range for hsv color space 
    YCrCb_mask = cv2.inRange(img_YCrCb, (0, 135, 85), (255,180,135)) 
    YCrCb_mask = cv2.morphologyEx(YCrCb_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))

    #merge skin detection (YCbCr and hsv)
    global_mask=cv2.bitwise_and(YCrCb_mask,HSV_mask)
    global_mask=cv2.medianBlur(global_mask, 5)
    global_mask = cv2.morphologyEx(global_mask, cv2.MORPH_OPEN, np.ones((3,3), np.uint8))


    HSV_result = cv2.bitwise_not(HSV_mask)
    YCrCb_result = cv2.bitwise_not(YCrCb_mask)
    global_result=cv2.bitwise_not(global_mask)
    print(global_mask.shape)

    return global_mask

def get_ground(image):
    height, width, _ = image.shape

    for y in range(height-1, -1, -1):
        count = 0
        for x in range(width):
            if image[y, x] == 255:
                count += 1
                if count >= 10:
                    return y  # Return the position of the first continuous 1s
            else:
                count = 0
    
    return None


def main(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, prev_frame = cap.read()
    head_positions = []
    frames = []

    while cap.isOpened():
        ret, cur_frame = cap.read()

        if not ret:
            break

        cur_frame = preprocess_frame(cur_frame)

        # circles = cv2.HoughCircles(cur_frame_gray, cv2.HOUGH_GRADIENT, 1, cur_frame_gray.shape[0]/8,
        #                     param1=100, param2=30, minRadius=1, maxRadius=20)

        # circles = np.uint16(np.around(circles))
        # for i in circles[0,:]:
        #     # draw the outer circle
        #     cv2.circle(cur_frame,(i[0],i[1]),i[2],(0,255,0),2)
        #     # draw the center of the circle
        #     cv2.circle(cur_frame,(i[0],i[1]),2,(0,0,255),3)


        # 피부색 검출
        skin_mask = detect_skin(cur_frame)
        cur_frame_hsv = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2HSV)
        
        v = cur_frame_hsv[:, :, 2]
        masked_v = cv2.bitwise_and(v, skin_mask)
        cur_frame_hsv[:, :, 2] = masked_v
        skin = cv2.cvtColor(cur_frame_hsv, cv2.COLOR_HSV2BGR)
        print(skin.shape)
        # Convert frames to grayscale
        # prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
        # curr_gray = cv2.cvtColor(cur_frame, cv2.COLOR_BGR2GRAY)
        
        # Estimate motion vector using 3 step search
        # motion_vector = three_step_search(prev_gray, curr_gray)
        
        # Update the position
        # x, y = prev_frame.shape[1] // 2, prev_frame.shape[0] // 2
        # x += motion_vector[0]
        # y += motion_vector[1]
        
        # # Draw a rectangle around the detected object
        # cv2.rectangle(prev_frame, (x-8, y-8), (x+8, y+8), (0, 255, 0), 2)
        
        # # Display the result
        # cv2.imshow('Result', prev_frame)


        # 결과 이미지 출력
        cv2.imshow('Original+high boosted', cur_frame)


        y_position = get_ground(skin)
        print(y_position)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
        skin = cv2.morphologyEx(skin, cv2.MORPH_CLOSE, kernel)
        cv2.line(skin, (0, y_position), (skin.shape[1], y_position), (0, 0, 255), 2)
        cv2.imshow('skin detection + draw circle_9', skin)

        # Update the previous frame
        prev_frame = cur_frame.copy()

        frames.append(cur_frame)
        if cv2.waitKey(100) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    pushup_count = count_pushups(head_positions)
    print(f"Total push-ups: {pushup_count}")

if __name__ == "__main__":
    video_path = "pushup_sample.mp4"
    main(video_path)

(360, 640)
(360, 640, 3)


ValueError: too many values to unpack (expected 2)

: 